# Dilithium - Glitches - Fixediter

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
SS_VER = 'SS_VER_2_1'

TIMEOUT = 510
N = 256
NUM_ITERS = 64
BETA = 78

In [2]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)
logging.getLogger('gurobipy.gurobipy').setLevel(logging.WARNING) # please be quiet gurobi
logging.getLogger().setLevel(logging.DEBUG + 1) # default logger should not be used anyways!
__LOGGER = logging.getLogger('sigglitches')

we append the path so that we can use the chipwhisperer library

In [3]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
if 'python-dilithium' not in sys.path:
    sys.path.append('python-dilithium')
if 'dilithium_solver' not in sys.path:
    sys.path.append('dilithium_solver')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../../software',
 'python-dilithium',
 'dilithium_solver']

In [4]:
import chipwhisperer as cw
import importlib
import json
import uuid
import numpy as np
import ipywidgets as widgets
from chipwhisperer.capture.targets import TargetIOError, TargetTimeoutError
from dilithium import Dilithium
import struct
d = Dilithium(2)
cw, d

(<module 'chipwhisperer' from '/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101/../../../software/chipwhisperer/__init__.py'>,
 <dilithium.Dilithium at 0x7fda087c26a0>)

In [5]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=$1 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=$2

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [6]:
%run "../../Setup_Scripts/Setup_Generic_Dilithium.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [7]:
fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

In [8]:
#cw.program_target(scope, prog, fw_path)

In [9]:
# found using
# target.filter_msgs_one_iter([struct.pack('!H', i) for i in range(10)])
msgs_no_retry = [b'\x00\x01', b'\x00\x02', b'\x00\x07', b'\x00\t']
the_msg = msgs_no_retry[0]

In [10]:
def dilithium2_polyz_unpack_settings():
    scope.default_setup()
    scope.cglitch_setup() # default_setup for clock glitching
    scope.clock.adc_src = "clkgen_x1" # we do not need x4 as we do not capture traces; maybe speedup?
    scope.adc.samples = 0 # maybe speedup?
    scope.adc.timeout = .000000002 # wtf why is this not working

def get_duration():
    dilithium2_polyz_unpack_settings()
    target.reboot_flush()
    scope.arm()
    target.loop(TIMEOUT)
    timed_out = scope.capture()
    assert not timed_out
    return scope.adc.trig_count;

def get_sig_correct() -> bytes:
    dilithium2_polyz_unpack_settings()
    target.sign(the_msg, timeout=TIMEOUT)
    return target.get_sig()

dilithium2_polyz_unpack_settings()
duration = get_duration()
iter_duration = duration // NUM_ITERS
sig_correct = get_sig_correct()
_, z_correct, _ = d._unpack_sig(sig_correct)
poly_correct = z_correct[0]

print(f'clock cycle duration all = {duration}')
print(f'clock cycle duration iteration = {iter_duration}')
z_correct

clock cycle duration all = 4698
clock cycle duration iteration = 73


array([[-100321,   65070,    9592, ...,  103227,   -3425,   17889],
       [-107453,  -81618,   31730, ...,  122362,  -45288,   23573],
       [  88900, -128866, -104785, ...,  -48187, -104331,   71746],
       [  -9268,   50775,   -6038, ..., -118320,  130605,  -62888]])

In [11]:
def get_iter(poly_candidate: np.ndarray, coeffs_per_iter: int = 4, beta: int = BETA) -> bool:
    """
    0 = all coeffs zero; 1 - 63 = some non faulted in the beginning rest zero;
    64 = no fault; None = not our fault model (i.e. faults within iters)
    """
    for i in range(0, NUM_ITERS + 1):
        # index of the first zero coefficient due to a fault
        split_index = i * coeffs_per_iter
        if np.all(np.abs(poly_candidate[split_index:]) <= beta):
            return i
    return None

In [12]:
glitch_spots = [2 * 62 - 73] # target iter #2]
width_start = 6.640625
width_stop = 6.640625
offset_start = 6.640625
offset_stop = 6.640625
msg_start = 0
msg_stop = 2 ** 16 -1
repeat = 1
step = 1
redo = 0

In [13]:
width_width = (width_stop - width_start + 1) / step
offset_width = (offset_stop - offset_start + 1) / step
ext_offset_width = len(glitch_spots)
print(width_width, offset_width, ext_offset_width)
normal_time = width_width * offset_width * ext_offset_width * redo
print(normal_time)
t = TIMEOUT * normal_time / 1000
print(f'Expected sig time: {t}s = {t/60}min = {t/3600}h')
t_all = t * 2.5 # what was that exactly again?
print(f'Expected sig + transfer time: {t_all}s = {t_all/60}min = {t_all/3600}h')

1.0 1.0 1
0.0
Expected sig time: 0.0s = 0.0min = 0.0h
Expected sig + transfer time: 0.0s = 0.0min = 0.0h


In [14]:
import chipwhisperer.common.results.glitch as glitch
MIN_ZEROS = 2 * d.n
zeros = 0
gc = glitch.GlitchController(groups=[f"perfect_{i}" for i in range(NUM_ITERS)] + ["success", "reset", "normal"], parameters=["width", "offset", "message"])
gc.display_stats()

IntText(value=0, description='perfect_0 count:', disabled=True)

IntText(value=0, description='perfect_1 count:', disabled=True)

IntText(value=0, description='perfect_2 count:', disabled=True)

IntText(value=0, description='perfect_3 count:', disabled=True)

IntText(value=0, description='perfect_4 count:', disabled=True)

...

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='message setting:', disabled=True, max=10.0, reado…

In [15]:
zero_widget = widgets.IntText(value=0, description="zeros (max)", disabled=True)
zero_widget

IntText(value=0, description='zeros (max)', disabled=True)

In [16]:
sigs_faulted = []
sigs_faulted_params = []

In [17]:
gc.set_range("offset", offset_start, offset_stop)
gc.set_range("width", width_start, width_stop)
gc.set_range("message", msg_start, msg_stop)
gc.set_global_step(step)
scope.glitch.repeat = repeat
target.reboot_flush()

b' \x00@\x04\x00\x00\x00\x0bb\x07boot ok\xc1\x00'

In [18]:
def one_try(scope, target, gc, message_int, sigs_faulted: list, sigs_faulted_params: list) -> bool:
    new_zeros = 0
    param_tuple = (scope.glitch.width, scope.glitch.offset, message_int)
    message = struct.pack('!H', message_int)
    scope.arm()
    try:
        target.sign(message, TIMEOUT)
        
        trigger_timeout = scope.capture()
        if trigger_timeout:
            gc.add("reset", param_tuple)
            __LOGGER.info('fail due to trigger timeout')
            target.reboot_flush()
            return new_zeros
        
    except TargetTimeoutError as e:
        gc.add("reset", param_tuple)
        __LOGGER.info(f'failed due to read timeout: {e}')
        target.reboot_flush()
        return new_zeros
    except TargetIOError as e:
        gc.add("reset", param_tuple)
        __LOGGER.info(f'failed due to TargetIOError error: {e}')
        target.reboot_flush()
        return new_zeros
    else:
        try:
            sig_candidate_packed = target.get_sig()
            _, z_candidate, _ = d._unpack_sig(sig_candidate_packed)
            
            if np.all(z_candidate == z_correct):
                gc.add("normal", param_tuple)
                return new_zeros
            i = get_iter(z_candidate[0])

            if i == NUM_ITERS:
                gc.add("success", param_tuple)
            elif 0 <= i < NUM_ITERS:
                gc.add(f"perfect_{i}", param_tuple)
                new_zeros = d.n - d._polyz_unpack_coeffs_per_iter * i
            else:
                __LOGGER.warning('Unexpected iteration!')
            
            __LOGGER.info("Glitched! Ext offset: {} Offset: {} Width: {} Message:{}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width, message))
            
            sigs_faulted.append(sig_candidate_packed)
            sigs_faulted_params.append(param_tuple)
        except TargetIOError as e:
            gc.add("reset", param_tuple)
            __LOGGER.info(f'Failed to read sig: {e}')
            target.reboot_flush()
    return new_zeros

In [19]:
start_time = time.time()
do_break = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    message = glitch_setting[2]
    for i in glitch_spots:
        scope.glitch.ext_offset = i
        for j in range(redo + 1):
            new_zeros = one_try(scope, target, gc, message, sigs_faulted, sigs_faulted_params)
            zeros += new_zeros
            zero_widget.value = zeros
        if zeros > MIN_ZEROS:
            do_break = True
    if do_break:
        break
end_time = time.time()
total_duration = end_time - start_time
print(f'total duration: {total_duration}s {total_duration/60}min {total_duration/3600}h')

INFO:sigglitches:Glitched! Ext offset: 51 Offset: 6.640625 Width: 6.640625 Message:b'\x00\x00'
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:Glitched! Ext offset: 51 Offset: 6.640625 Width: 6.640625 Message:b'\x00\x02'
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:Glitched! Ext offset: 51 Offset: 6.640625 Width: 6.640625 Message:b'\x00\x04'


total duration: 4.672922849655151s 0.07788204749425252min 0.0012980341249042087h


In [20]:
result_data = {
    "sig_correct": list(sig_correct),
    "glitch_spots": list(glitch_spots),
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
    "redo": redo,
    "repeat": repeat,
    "step": step,
    "sigs_faulted": list(map(lambda s: list(s), sigs_faulted)),
    "sigs_faulted_param_descripton": "0 = ext_offset, 1 = offset, 2 = width",
    "sigs_faulted_params": sigs_faulted_params
}
with open(f'iter2{uuid.uuid4()}.json', 'w') as f:
    json.dump(result_data, f)
#json.dumps(result_data)

In [21]:
sigs_faulted_unpacked = list(map(lambda sig_packed: d._unpack_sig_full(sig_packed), sigs_faulted))
from dilithium_solver.signature import Signature, calculate_c_matrix_np
from dilithium_solver.recover_s_1_entry import recover_s_1_entry
from dilithium_solver.parameters import Parameters

params = Parameters.get_nist_security_level(d.nist_security_level)

sigs = list(map(
    lambda sig_faulted: Signature(
        sig_faulted[1],
        sig_faulted[0],
        calculate_c_matrix_np(sig_faulted[0], params)
    ), sigs_faulted_unpacked))
s_1_entry_index = 0

# what was the sk again?
d.pseudorandombytes_seed(b'attack-shuffling-countermeasure-keypair')
pk, sk = d.keypair()

s_1 = d._unpack_sk(sk)[4]
timeout = 60
threshold = d.beta
for i in range(len(sigs_faulted_unpacked[0][1])): # long version of saying "l"
    result = recover_s_1_entry(sigs, i, s_1, params, 142387, timeout, threshold) # this number is not relevant
    print(result)
    break

INFO:root:Before filter: 768; after filter: 753; delta: 15; threshold: 78
INFO:root:Total filtered: 753; true: 142387; false: -141634; false-positive rate: -18809.29614873838%; probability: nan


Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-07
Set parameter TimeLimit to value 60
Set parameter Threads to value 4


INFO:root:Starting recovery of an entry. Will timeout after 60s. Current datetime is: 2022-12-14 02:16:55.337604


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-6850K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 4 threads

Optimize a model with 1506 rows, 1009 columns and 60240 nonzeros
Model fingerprint: 0x7f91e71b
Variable types: 0 continuous, 1009 integer (753 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+05, 1e+05]
Found heuristic solution: objective 8.0000000
Presolve time: 0.04s
Presolved: 1506 rows, 1009 columns, 60240 nonzeros
Variable types: 0 continuous, 1009 integer (753 binary)
Found heuristic solution: objective 41.0000000

Root relaxation: objective 7.530000e+02, 674 iterations, 0.16 seconds (0.24 work units)


INFO:root:Candidate is correct with same sign



    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     753.0000000  753.00000  0.00%     -    0s


INFO:root:Candidate is correct with same sign



Explored 1 nodes (674 simplex iterations) in 0.25 seconds (0.29 work units)
Thread count was 4 (of 6 available processors)

Solution count 3: 753 41 8 

Solve interrupted
Best objective 7.530000000000e+02, best bound 7.530000000000e+02, gap 0.0000%

User-callback calls 184, time in user-callback 0.01 sec


INFO:root:We used 753.0 zero coefficients in y to recover the secret key.
INFO:root:Recovery took 0.2575080394744873 seconds.
INFO:root:Successfully recovered s_1 entry!


(array([ 0.,  1.,  2., -2., -1., -1., -2.,  2., -2.,  2.,  2., -1.,  2.,
        1.,  0.,  2., -2.,  2.,  1., -1.,  0., -2.,  0.,  2.,  1., -1.,
       -2., -1.,  0.,  1., -2.,  1.,  0.,  2.,  0.,  1.,  0.,  0., -2.,
       -2., -2.,  1., -2.,  0.,  0., -2.,  0., -1., -2., -1.,  2.,  2.,
        1.,  2.,  1.,  0.,  1., -2.,  0., -2.,  1., -2.,  1., -2., -2.,
        2., -1.,  0., -2., -1.,  1.,  1.,  1.,  2., -2.,  1.,  0., -1.,
        0.,  2.,  1.,  0.,  2.,  1.,  2., -1.,  2.,  1.,  2.,  0., -1.,
       -1.,  2., -1., -1.,  1., -1.,  2., -1.,  1., -1.,  0., -1.,  1.,
        0.,  0., -2.,  0.,  2.,  1.,  1.,  0.,  2., -1., -1.,  2.,  1.,
        2.,  0., -1.,  1.,  0., -2.,  2., -2., -1., -2.,  1.,  0.,  0.,
        0.,  0.,  1.,  1., -1., -1.,  2., -1.,  0.,  2., -1., -2.,  2.,
        2.,  2., -1.,  0., -2.,  1.,  2.,  0.,  1.,  1.,  1.,  0., -1.,
        2.,  0., -2., -2.,  1., -1.,  1.,  2., -1.,  0.,  2.,  0., -2.,
       -1., -1.,  2.,  2.,  2., -1.,  0., -2.,  0.,  1., -1., -

In [22]:
scope.dis()
target.dis()